In [1]:
import shapeless._
import shapeless.syntax.singleton._
import shapeless.labelled._
import shapeless.ops.record.Remover

import shapeless._

import shapeless.syntax.singleton._

import shapeless.labelled._

import shapeless.ops.record.Remover

In [1]:
// Talk about Phantom types, Literal Types, FieldType, Records, LabelledGeneric

In [1]:
// We'd like to have a merge function, that we want to be able to use it like:
// webviewAdId merge nativeAdId

In [2]:
trait Merger[L <: HList, R <: HList] extends DepFn2[L, R] { 
    type Out <: HList 
    def apply(l: L, r: R): Out
}

defined trait Merger

In [3]:
trait LowPriorityMerger {
    type Aux[L <: HList, R <: HList, Out0 <: HList] = Merger[L, R] { type Out = Out0}
    implicit def hlist1Merger1[H, T <: HList, R <: HList](implicit mt:Merger[T, R]):Aux[H :: T, R, H :: mt.Out] =
      new Merger[H :: T, R] {
          type Out = H :: mt.Out
          def apply(l : H :: T, r : R): Out = l.head :: mt(l.tail, r)
      }
}

defined trait LowPriorityMerger

In [4]:
object Merger extends LowPriorityMerger {

    implicit def hnilMerger[R <: HList]:Aux[HNil, R, R] = 
      new Merger[HNil, R] {
          type Out = R
          def apply(l: HNil, r: R): R = r
      }
    
    // L - H :: T - (K, V) :: T
    implicit def hlistMerger2[K, V, T <: HList, R <: HList , RT <: HList](
        implicit rm : Remover.Aux[R, K, (V, RT)],
        mt: Merger[T, RT]
    ):Aux[FieldType[K, V] :: T, R, FieldType[K, V] :: mt.Out] = 
    
    new Merger[FieldType[K, V] :: T, R] {
        type Out = FieldType[K, V] :: mt.Out
        def apply(l: FieldType[K, V] :: T, r: R):Out = {
            val (rv, rr) = rm(r)
            val upd = field[K](rv)
            upd :: mt(l.tail, rr)
        }
    }
}

defined object Merger

In [5]:
trait CaseClassMerge[L, R] {
    def merge(l:L, r:R):L
}
object CaseClassMerge {
    import ops.record.Merger
    // summoner
    def apply[L, R](implicit merge: CaseClassMerge[L, R]):CaseClassMerge[L, R] = merge
    
    implicit def mkCCMerge[L, R, RL <: HList, RR <: HList](
        implicit lgen : LabelledGeneric.Aux[L, RL],
        rgen : LabelledGeneric.Aux[R, RR],
        merger: Merger.Aux[RL, RR, RL]
    ) : CaseClassMerge[L, R] = new CaseClassMerge[L, R] {
        def merge(l: L, r: R): L = lgen.from(merger(lgen.to(l), rgen.to(r)))
    }
}

defined trait CaseClassMerge
defined object CaseClassMerge

In [6]:
object mergesyntax {
    import CaseClassMerge._
    implicit class MergeSyntax[L](l: L) {
        def merge[R](r:R)(implicit test: CaseClassMerge[L, R]):L = test.merge(l, r)
    }
}

defined object mergesyntax

In [7]:
// Merge two case classes
case class WebviewAdId(id: String, priceType: Int, isWebview: Boolean)
case class NativeAdId(priceType:Int, id:String, isWebview:Boolean)
val webviewAdId = WebviewAdId("wbi", 1, true)
val nativeAdId = NativeAdId(2, "nbi", false)
import mergesyntax._

webviewAdId merge nativeAdId

defined class WebviewAdId
defined class NativeAdId
webviewAdId: WebviewAdId = WebviewAdId("wbi", 1, true)
nativeAdId: NativeAdId = NativeAdId(2, "nbi", false)
import mergesyntax._


res6_5: WebviewAdId = WebviewAdId("nbi", 2, false)